# Create tick-tack-toe agent by "Sarsa"
## Reference

Reinforcement Learning: An Introduction (Richard S. Sutton and Andrew G. Barto)
  - chapter 6.4 Sarsa: On-Policy TD Control
  - https://webdocs.cs.ualberta.ca/~sutton/book/ebook/node64.html

## algorithm
<img src="https://webdocs.cs.ualberta.ca/~sutton/book/ebook/pseudotmp8.png" />
https://webdocs.cs.ualberta.ca/~sutton/book/ebook/pseudotmp8.png

## Prepare functions to play tick-tack-toe

In [1]:
def visualize_board(state):
    board = [-1 for i in range(9)]
    for i in range(2):
        for j in range(9):
            if state[i] >> j &1 == 1:
                board[j] = i
    icon_map = {-1: "-", 0:"o", 1:"x"}
    board = map(lambda i: icon_map[i], board)
    for i in range(3):
        print "%s %s %s" % tuple(board[i*3:(i+1)*3])

# return empty cell position on the board
def possible_actions(state):
    board = state[0] | state[1]
    def add(ary, pos):
        if (board >> pos)&1 == 0:
            ary.append(1<<pos)
        return ary
    return reduce(add, range(9), [])

def is_terminated(player_board):
    bin2i = lambda b: int(b, 2)
    line_horizon = any([player_board & mask == mask for mask in map(bin2i, ['000000111', '000111000', '111000000'])])
    line_vertical = any([player_board & mask == mask for mask in map(bin2i, ['001001001', '010010010', '100100100'])])
    line_diagonal = any([player_board & mask == mask for mask in map(bin2i, ['100010001', '001010100'])])
    return line_horizon | line_vertical | line_diagonal

def is_draw(state):
    return len(possible_actions(state))==0

def calc_reward(state):
    first_player_board, second_player_board = state
    if (is_terminated(first_player_board)):
        return 1
    elif(is_terminated(second_player_board)):
        return -1
    else:
        return 0

def apply_action(state, action):
    first_player_board ,second_player_board = state
    count_move = lambda : bin(first_player_board | second_player_board).count("1")
    if count_move()%2==0:
        first_player_board |= action
    else:
        second_player_board |= action
    return first_player_board, second_player_board

def play_a_game():
    state = (0,0)
    visualize_board(state)
    while not any(map(is_terminated, state) + [is_draw(state)]):
        act = int(raw_input("action => %s" % possible_actions(state)))
        state = apply_action(state, act)
        visualize_board(state)

## Prepare functions for GPI 

In [7]:
# Play 1 game and return Q-value (state, reward pair) array watched in the game
def gen_episode(Q, debug=False):
    state = (0, 0)
    episode = []
    while not any(map(is_terminated, state) + [is_draw(state)]):
        action = policy(Q, state, debug=debug)
        episode.append((state, action))
        _, state = transition(state, action)
        if debug: visualize_board(state)
    return episode, calc_reward(state)

import math, random
# Epsiron-greedy
def policy(Q, current_state, eps=0.1, debug=False):
    do_random = lambda : random.random() < eps
    transition_curry = lambda action: transition(current_state, action)
    Q_value_curry = lambda action: Q[current_state[0]][current_state[1]][int(math.log(action,2))]
    if do_random():
        if debug: print "do random"
        return random.choice(possible_actions(current_state))
    else:
        if debug: print "do greedy"
        Q_value_for_actions = map(Q_value_curry, possible_actions(current_state))
        best_act_idx = Q_value_for_actions.index(max(Q_value_for_actions))
        return possible_actions(current_state)[best_act_idx]

# return  next state and reward after passed action is applied
def transition(state, action):
    next_state =  apply_action(state, action)
    reward = calc_reward(next_state)
    return reward, next_state

In [19]:
# policy evaluation => policy improvement process
def Sarsa(Q, alpha=0.1, gamma=0.1):
    episode, reward = gen_episode(Q)
    for state, action in episode:
        next_reward, next_state = transition(state, action)
        is_terminal_state = any(map(is_terminated, next_state) + [is_draw(next_state)])
        next_action = policy(Q, next_state) if not is_terminal_state else 1   # How to fetch next_action from terminal state??
        s1, s2, a = state[0], state[1], int(math.log(action,2))
        ns1, ns2, na = next_state[0], next_state[1], int(math.log(next_action, 2))
        Q[s1][s2][a]= Q[s1][s2][a] + alpha * ( next_reward + gamma * Q[ns1][ns2][na] -  Q[s1][s2][a])

def display_Q_value(Q, state):
    Q_value_curry = lambda action: Q[state[0]][state[1]][int(math.log(action,2))]
    Q_values =map(Q_value_curry, [1<<n for n in range(9)])
    max_len = max([len(str(val)) for val in Q_values])
    for i in range(3):
        line = []
        for j in range(3):
            pos = 3*i+j
            board = state[0] | state[1]
            line.append(str(Q_values[pos]) if (board>>pos)&1 == 0 else "x")
        print "%s %s %s" % tuple([elem.rjust(max_len, '0') for elem in line])

## Start learning !! Iterate GPI process for 50000 times

In [20]:
Q_table = [[[0 for a in range(9)] for j in range(2**9)] for i in range(2**9)]

# Improve agent by iterating GPI process
for i in range(50000):
    Sarsa(Q_table)
display_Q_value(Q_table, (0,0))

8.19680241422e-05 6.57291627587e-05 7.94095934152e-05
6.61682847706e-05 07.6603597521e-05 6.44298268849e-05
7.90297811993e-05 6.74716961773e-05 6.97592852368e-05


## Prepare functions to play with agent through console

In [22]:
def is_next_player_agent(state):
    first_player_board ,second_player_board = state
    count_move = lambda : bin(first_player_board | second_player_board).count("1")
    return count_move() % 2 == 0

def play_with_agent():
    state = (0,0)
    while not any(map(is_terminated, state) + [is_draw(state)]):
        action, player = None, None
        if is_next_player_agent(state):
            display_Q_value(Q_table, state)
            action = policy(Q_table, state)
            player = "agent"
        else:
            action = int(raw_input("action => %s" % possible_actions(state)))
            player = "you"
        state = apply_action(state, action)
        visualize_board(state)
        print "player : %s, action : %d" % (player, int(math.log(action, 2)))

## Let's play tick-tack-toe with our agent

In [23]:
play_with_agent()

8.19680241422e-05 6.57291627587e-05 7.94095934152e-05
6.61682847706e-05 07.6603597521e-05 6.44298268849e-05
7.90297811993e-05 6.74716961773e-05 6.97592852368e-05
o - -
- - -
- - -
player : agent, action : 0
action => [2, 4, 8, 16, 32, 64, 128, 256]2
o x -
- - -
- - -
player : you, action : 1
0000000000000000x 0000000000000000x 8.52016341286e-05
00.00790387425769 00.00779503256187 7.91517850475e-05
00.00891134215165 08.2244066792e-05 00.00709218319692
o x -
- - -
o - -
player : agent, action : 6
action => [4, 8, 16, 32, 128, 256]16
o x -
- x -
o - -
player : you, action : 4
000000000000000x 000000000000000x 0.00844502959246
00000000000001.0 000000000000000x 0.00772545428489
000000000000000x 00.0086562767804 0.00819604536569
o x -
o x -
o - -
player : agent, action : 3


`>>> Good!! Agent seems understanding how to win!!`

In [24]:
play_with_agent()

8.19680241422e-05 6.57291627587e-05 7.94095934152e-05
6.61682847706e-05 07.6603597521e-05 6.44298268849e-05
7.90297811993e-05 6.74716961773e-05 6.97592852368e-05
o - -
- - -
- - -
player : agent, action : 0
action => [2, 4, 8, 16, 32, 64, 128, 256]8
o - -
x - -
- - -
player : you, action : 3
0000000000000000x 00.00861475209802 0.000323658975269
0000000000000000x 0.000908143497353 01.2664670074e-07
06.6676853621e-05 1.59608407136e-07 1.96507945238e-05
o o -
x - -
- - -
player : agent, action : 1
action => [4, 16, 32, 64, 128, 256]4
o o x
x - -
- - -
player : you, action : 2
000000000000000x 000000000000000x 000000000000000x
000000000000000x 0.00945520817625 00000000000000.0
00000000000000.0 00000000000000.0 00000000000000.0
o o x
x o -
- - -
player : agent, action : 4
action => [32, 64, 128, 256]128
o o x
x o -
- x -
player : you, action : 7
000000000000000x 000000000000000x 000000000000000x
000000000000000x 000000000000000x 0.00178408605053
0000000000000000 000000000000000x 00000000000

`>>> Woops... Agent made a silly mistake :(`

`>>> It seems that 50000 GPI iteration was too small (Most of states are never experienced)`